In [1]:
import csv
import os
import nltk 
import re                               
import string                   
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
import pandas as pd
punc = string.punctuation
from collections import Counter

In [2]:
class autocorrect :
    
    def __init__(self):
        pass

    def get_data_file(self, path = pd.read_csv(r'F:\7sabat\GP\New folder\speech-recognition-for-people-with-dysarthria\English_auto_correct_and_TSS\data\UAspeechVocab.csv')):
        self.corpus = path['label']
        return self.corpus
        
    def get_words_cleaned(self):
        updated_corpus = []
        corpus = self.get_data_file()
        for word in corpus:
            updated_corpus.append(word.strip("\n").lower())
        return updated_corpus
    
    def get_counts(self):
        corpus = self.get_words_cleaned()
        words_counts = Counter(corpus)
        return words_counts
    
    def probability(self,word):
        words_counts = self.get_counts()
        N = sum(words_counts.values())
        return words_counts[word]/N
    
    def if_known(self,words):
        words_counts = self.get_counts()
        revelant = []
        for w in words:
            if w in words_counts:
                revelant.append(w)
        return revelant
    
    def edits_1(self,word):
        letters_english = 'abcdefghijklmnopqrstuvwxyz'
        letters_arbic = 'ابتثجحخدذرزسشصضطظعغفقكلمنهويةىء'
        splits = [(word[:i],word[i:]) for i in range(len(word)+1)]
        deletes = [left + right[1:] for left,right in splits if right]
        inserts = [left + center + right for left , right in splits for center in letters_english]
        replaces = [left + center + right[1:] for left , right in splits if right for center in letters_english]
        transposes = [left + right[1] + right[0] +right[2:] for left , right in splits if len(right)>1]
        return set(deletes + inserts + replaces + transposes)

    def edits_2(self,word):
        for edit1 in self.edits_1(word):
            for edit2 in self.edits_1(edit1):
                return edit2
            
    def candidates(self,word):
        return ( self.if_known(self.edits_1(word)) or self.if_known(self.edits_2(word)) or self.if_known([word]) or [word]) 
    def best_correction(self,word):
        candidate = self.candidates(word)
        return max(candidate,key = self.probability)

In [3]:
correct = autocorrect()

In [4]:
correct.probability('upward')

0.0038801680300685386

In [6]:
correct.best_correction('carot')

'carrot'

In [7]:
correct.candidates('carot')

['carrot']

In [8]:
correct.best_correction('upword')

'upward'

In [9]:
correct.candidates('upword')

['upward']

In [11]:
correct.candidates('tha')

['the', 'than', 'that']

In [12]:
correct.best_correction('tha')

'the'